In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import argparse
from tensorboardX import SummaryWriter
import numpy as np
import random
import sys
import os

# currentdir = os.path.dirname(os.path.realpath(__file__))
# parentdir = os.path.dirname(os.path.dirname(currentdir))
# sys.path.append(parentdir)

from utils.Empirical.architectures import ARCHITECTURES
from utils.Empirical.datasets import DATASETS

from utils.Empirical.utils_ensemble import AverageMeter, accuracy, test, copy_code, requires_grad_, evaltrans
from utils.Empirical.datasets import get_dataset
from utils.Empirical.architectures import get_architecture
from train.Empirical.trainer import Naive_Trainer

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = get_dataset('cifar10', 'train')
test_dataset = get_dataset('cifar10', 'test')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128,
                              num_workers=4 )
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=128,
                             num_workers=4 )

Files already downloaded and verified
Files already downloaded and verified


In [61]:
samp_cont = 0
mu , sigma = 0,0
for i, (inputs, targets) in enumerate(train_loader):
    # inputs, targets =inputs.to(device), targets.to(device)          
    batch_size = inputs.size(0)
    pixel = inputs.size(1)*inputs.size(2)*inputs.size(3)
    idx_list = np.arange(batch_size)

    samp_list = np.random.choice(idx_list, size= np.int64( np.ceil(0.02*batch_size) ), replace=False,p = None)
    samp_num = len(samp_list)
    
    # for idx in samp_list:
    curr_mu = torch.mean(inputs[samp_list])
    curr_sigma = torch.var(inputs[samp_list]) 
    
    mu = (mu*samp_cont + curr_mu*samp_num)/(samp_cont + samp_num)
    sigma = (sigma*samp_cont + curr_sigma*samp_num)/(samp_cont + samp_num)
    samp_cont = samp_cont + samp_num
lower_b =np.int64(np.ceil( (mu - 2*sigma)*255 ))
upper_b =np.int64(np.ceil( (mu + 2*sigma)*255 ))

print(lower_b,upper_b)


67 144
